### Q1a

In [ ]:
class Cart:
    def __init__(self, a, b):
        self.a, self.b = a, b
        self.i, self.j = 0, 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.i == len(self.a):
            raise StopIteration
        
        t = (self.a[self.i], self.b[self.j])

        self.j += 1
        if self.j == len(self.b):
            self.j = 0
            self.i += 1
        
        return t

for i in Cart([1,2], ["a", "b"]):
    print(i)

### Q1b

In [ ]:
def cart(a, b):
    for i in a:
        for j in b:
            yield (i,j)

for i in cart([1,2], ["a", "b"]):
    print(i)

### Q2

In [ ]:
import os

def sortuniq(inpfilepath, outfilepath):
    os.system("/usr/bin/sort < " + inpfilepath + " | /usr/bin/uniq > " + outfilepath)

sortuniq("/Users/mehmetruchanyavuzdemir/Desktop/python-445/notebooks/input.txt", 
         "/Users/mehmetruchanyavuzdemir/Desktop/python-445/notebooks/output.txt")

In [ ]:
from subprocess import *

def sortuniq(inpfilepath, outfilepath):
    ifile = open(inpfilepath, "r")
    ofile = open(outfilepath, "w")
    sort_process = Popen("/usr/bin/sort", stdin=ifile, stdout=PIPE)
    uniq_process = Popen("/usr/bin/uniq", stdin=sort_process.stdout, stdout=ofile)

    uniq_process.wait()

In [ ]:
sortuniq("/Users/mehmetruchanyavuzdemir/Desktop/python-445/notebooks/input.txt", 
         "/Users/mehmetruchanyavuzdemir/Desktop/python-445/notebooks/output.txt")

### Q3a

In [2]:
import re

class Logger:
    """
        Returns a new line of log. Assume already implemented.
    """
    def nextline(self):
        return "144.12.1.121 tried X action at 02/12/2015"


class Highlighter: # Base Class  
    def __init__(self, t):
        self.obj = t

    def __getattr__(self, name):
        return getattr(self.obj, name)
    

class HLDate(Highlighter):
    """
        Gets next line of log from the member object and return with replacing all date occurrences of 
        02/12/2015 into **02/12/2015**
    """
    def nextline(self):
        return re.sub("02/12/2015", "**02/12/2015**", self.obj.nextline())


class HLIP(Highlighter):
    """
        Gets next line of log from the member object and return with replacing all IP occurrences like
        144.12.1.121 into 144.12.*.*
    """
    def nextline(self):
        return re.sub("144.12.1.121", "144.12.*.*", self.obj.nextline())

### Q3b

Decorator Pattern.

In [ ]:
a = Logger()
b = HLIP(Logger())
c = HLDate(Logger())
d = HLIP(HLDate(Logger()))

print(a.nextline())
print(b.nextline())
print(c.nextline())
print(d.nextline())

In [ ]:
a = HLIP(HLDate(Logger()))
b = HLIP(Logger())

# I = a.nextline()
# while I != '':
#     print(I)
#     I = a.nextline()

### Q4

In [ ]:
from threading import *

class GameRoom:
    def __init__(self):
        self.nempty = 4
        self.ninside = 0

        self.lock = Lock()
        self.start = Condition(self.lock)
        self.turn = Condition(self.lock)

    def enter(self, name):
        with self.lock:
            while self.ninside == 4:
                self.turn.wait()

            print(name + " has entered!")
            self.ninside += 1
            self.nempty -= 1

            if self.nempty == 0:
                self.start.notify_all()
                print("Game has started!")
            else:
                while self.ninside < 4:
                    self.start.wait()


    def exit(self, name):
        with self.lock:
            print(name + " has left!")
            self.nempty += 1
            
            if self.nempty == 4:
                self.ninside = 0
                self.turn.notify_all()
                print("Game is over!")

In [ ]:
import time
import random

def player(room,name):
	time.sleep(random.random()/100)
	room.enter(name)
	time.sleep(random.random()/100)
	room.exit(name)


gr = GameRoom()

t = []
for i in range(52):
	t.append(Thread(target=player, args=(gr, str(i))))

for tr in t:
	tr.start()

for tr in t:
	tr.join()

### Q5

In [ ]:
from multiprocessing import *
from socket import *
import sys

def parsecommand(line): # Parses client input for convenience
    (req, num) = line.decode().rstrip('\n').split(' ')
    return (req, int(num))

class Server(Process): # Per process code is here
    def __init__(self, ns, pricelist, is_sold, lock):
        self.ns = ns
        self.pricelist = pricelist
        self.is_sold = is_sold
        self.lock = lock
        Process.__init__(self)

    def buy(self, i):
        with self.lock:
            if self.is_sold[i]:
                self.ns.send(b"SOLD\n")
            else:
                self.is_sold[i] = True
                self.ns.send(b"OK\n")

    def price(self, i):
        self.ns.send((str(self.pricelist[i]) + "\n").encode())
    
    def run(self):
        while True:
            line = self.ns.recv(1024)
            (req, i) = parsecommand(line)
            if (req == "BUY"):
                self.buy(i)

            elif (req == "PRICE"):
                self.price(i)


def startserver(host, port, pricelist):
    s = socket(AF_INET, SOCK_STREAM)
    s.bind((host, port))
    s.listen(10)

    n = len(pricelist)

    is_sold = Array('b', n)
    for i in range(n):
        is_sold[i] = False

    lock = Lock()

    while True:
        ns, peer = s.accept()
        print("Accepted: ", peer)
        p = Server(ns, pricelist, is_sold, lock)
        p.start()

if __name__ == '__main__':
    startserver('0.0.0.0', int(sys.argv[1]), list(map(float, sys.argv[2].split(','))))
    # python3 solution.py 1000 10,6,9,3
    # nc 127.0.0.1 1000